# GAN Project: Generative Dog Images

In [1]:
# Import necessary libraries
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, UpSampling2D, Reshape, LeakyReLU
from tensorflow.keras.optimizers import Adam

# Set random seed for reproducibility
np.random.seed(42)

# Set the directory for the dataset
dataset_dir = '/path/to/dataset'

In [ ]:
# Function to load and display an example image from the dataset
def load_image(file_path):
    img = load_img(file_path)
    img_array = img_to_array(img)
    return img_array

# Example: Load and visualize a sample image
example_image_path = os.path.join(dataset_dir, 'sample_image.jpg')
example_image = load_image(example_image_path)

plt.imshow(example_image.astype('uint8'))
plt.title('Sample Image')
plt.axis('off')
plt.show()

# Display basic information about the dataset
print(f'Dataset directory: {dataset_dir}')
print(f'Number of images in the dataset: {len(os.listdir(dataset_dir))}')

In [ ]:
# Preprocess the images: normalize and resize
def preprocess_image(image, target_size=(64, 64)):
    img = load_img(image, target_size=target_size)
    img_array = img_to_array(img)
    img_array = img_array / 255.0  # Normalize pixel values to [0, 1]
    return img_array

# Preprocess all images in the dataset
processed_images = []
for img_file in os.listdir(dataset_dir):
    img_path = os.path.join(dataset_dir, img_file)
    processed_img = preprocess_image(img_path)
    processed_images.append(processed_img)

processed_images = np.array(processed_images)
print(f'Processed images shape: {processed_images.shape}')

In [ ]:
# Build the generator model
def build_generator():
    model = Sequential()
    
    model.add(Dense(256 * 8 * 8, input_dim=100))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Reshape((8, 8, 256)))
    model.add(UpSampling2D())
    
    model.add(Conv2D(256, kernel_size=3, padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(UpSampling2D())
    
    model.add(Conv2D(128, kernel_size=3, padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(UpSampling2D())
    
    model.add(Conv2D(64, kernel_size=3, padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    
    model.add(Conv2D(3, kernel_size=3, padding='same', activation='tanh'))
    
    return model

# Build the discriminator model
def build_discriminator():
    model = Sequential()
    
    model.add(Conv2D(64, kernel_size=3, strides=2, input_shape=(64, 64, 3), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(128, kernel_size=3, strides=2, padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(256, kernel_size=3, strides=2, padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    
    return model

# Compile the models
generator = build_generator()
discriminator = build_discriminator()

discriminator.compile(optimizer=Adam(learning_rate=0.0002), loss='binary_crossentropy', metrics=['accuracy'])

# Display model summaries
generator.summary()
discriminator.summary()